# Session 07: Practice API requests

We will use ESIOS API to practice requests, reading documentation and handling responses.

### Exercise 1: Create a function that builds the headers for the requests

In [ ]:
import os
from dotenv import load_dotenv

def create_headers():

    load_dotenv()

    api_token = os.getenv("API_KEY_PYTHON_CLASS")

    headers = {
        'Accept': 'application/json; application/vnd.esios-api-v1+json',
        'Content-Type': 'application/json',
        'x-api-key': f'{api_token}'
    }

    return headers

create_headers()

In [3]:
load_dotenv??

Signature:
load_dotenv(
    dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None,
    stream: Union[IO[str], NoneType] = None,
    verbose: bool = False,
    override: bool = False,
    interpolate: bool = True,
    encoding: Union[str, NoneType] = 'utf-8',
) -> bool
Source:   
def load_dotenv(
    dotenv_path: Optional[StrPath] = None,
    stream: Optional[IO[str]] = None,
    verbose: bool = False,
    override: bool = False,
    interpolate: bool = True,
    encoding: Optional[str] = "utf-8",
) -> bool:
    """Parse a .env file and then load all the variables found as environment variables.

    Parameters:
        dotenv_path: Absolute or relative path to .env file.
        stream: Text stream (such as `io.StringIO`) with .env content, used if
            `dotenv_path` is `None`.
        verbose: Whether to output a warning the .env file is missing.
        override: Whether to override the system environment variables with the variables
            from the `.e

### Exercise 2: Create a function that returns the indicators list

In [7]:
import requests 
import pandas as pd

def get_indicators_list(url, headers):
    '''
    This function returns the list of indicators available in the API
    '''

    try:
        response = requests.get(url, headers = headers)
    except:
        print('Some Error')
    
    return pd.DataFrame(response.json()['indicators'])

url = 'https://api.esios.ree.es/indicators/'
headers = create_headers()

df_indicators_list = get_indicators_list(url, headers)

df_indicators_list.head()

,name,description,short_name,id
0,Generación programada PBF Hidráulica UGH,"<p>Es el programa de energía diario, con desgl...",Hidráulica UGH,1
1,Generación programada PBF Hidráulica no UGH,"<p>Es el programa de energía diario, con desgl...",Hidráulica no UGH,2
2,Generación programada PBF Turbinación bombeo,"<p>Es el programa de energía diario, con desgl...",Turbinación bombeo,3
3,Generación programada PBF Nuclear,"<p>Es el programa de energía diario, con desgl...",Nuclear,4
4,Generación programada PBF Hulla antracita Anex...,"<p>Es el programa de energía diario, con desgl...",Hulla antracita RD 134/2010,5


### Exercise 3: Create a function that returns the data for a single indicator

In [18]:
def single_indicator(indicator_id, url, headers):
    '''
    This function returns the information of a single indicator
    '''

    try:
        response = requests.get(f'{url}{indicator_id}', headers = headers)
    except:
        print('Some Error')
    
    return pd.DataFrame(response.json())

df_single_indicator = single_indicator(4, url, headers)

df_single_indicator

,indicator
composited,False
disaggregated,False
geos,"[{'geo_id': 8741, 'geo_name': 'Península'}]"
id,4
magnitud,"[{'name': 'Energía', 'id': 13}]"
name,Generación programada PBF Nuclear
short_name,Nuclear
step_type,step
tiempo,"[{'name': 'Hora', 'id': 4}]"
values,"[{'value': 6737.5, 'datetime': '2025-01-28T00:..."


### Exercise 4: Create a function that returns the data of a single indicator also passing parameters

In [21]:
def single_indicator_with_params(indicator_id, params, url, headers):
    '''
    This function returns the information of a single indicator
    '''

    try:
        response = requests.get(f'{url}{indicator_id}', params = params, headers = headers)
    except:
        print('Some Error')
    
    return pd.DataFrame(response.json())

params = {
    'start_date': '2025-01-23T23:00:00Z',
    'end_date': '2025-12-24T22:59:59Z'
}

df_single_indicator_with_params = single_indicator_with_params(4, params, url, headers)

df_single_indicator_with_params

,indicator
composited,False
disaggregated,False
geos,"[{'geo_id': 8741, 'geo_name': 'Península'}]"
id,4
magnitud,"[{'name': 'Energía', 'id': 13}]"
name,Generación programada PBF Nuclear
short_name,Nuclear
step_type,step
tiempo,"[{'name': 'Hora', 'id': 4}]"
values,"[{'value': 7092.9, 'datetime': '2025-01-24T00:..."


### Exercise 5: Create a function that takes a column name containing timestamp data, and then creates several other calendar columns

In [24]:
df = pd.DataFrame({'datetime': {'2025-01-23T23:00:00Z': 0, '2025-01-23T23:30:00Z': 1, '2025-01-23T23:59:59Z': 2}, 'value': {'2025-01-23T23:00:00Z': 0.0, '2025-01-23T23:30:00Z': 0.0, '2025-01-23T23:59:59Z': 0.0}})
df

,datetime,value
2025-01-23T23:00:00Z,0,0.0
2025-01-23T23:30:00Z,1,0.0
2025-01-23T23:59:59Z,2,0.0


In [23]:
def create_calendar_features(column_name, df):
    '''
    This function creates the calendar features of a dataframe
    '''
    #if df[column_name].dtype == 'object':
    df[column_name] = pd.to_datetime(df[column_name])

    df['year'] = df[column_name].dt.year
    df['month'] = df[column_name].dt.month
    df['day'] = df[column_name].dt.day
    df['hour'] = df[column_name].dt.hour

    return df

create_calendar_features('datetime', df)

,datetime,value,year,month,day,hour
2025-01-23T23:00:00Z,1970-01-01 00:00:00.000000000,0.0,1970,1,1,0
2025-01-23T23:30:00Z,1970-01-01 00:00:00.000000001,0.0,1970,1,1,0
2025-01-23T23:59:59Z,1970-01-01 00:00:00.000000002,0.0,1970,1,1,0


### Exercise 6: Using the indicators list, create a function that looks for a specific word and returns the indicators that contain that word

In [25]:
df_indicators_list.head()

,name,description,short_name,id
0,Generación programada PBF Hidráulica UGH,"<p>Es el programa de energía diario, con desgl...",Hidráulica UGH,1
1,Generación programada PBF Hidráulica no UGH,"<p>Es el programa de energía diario, con desgl...",Hidráulica no UGH,2
2,Generación programada PBF Turbinación bombeo,"<p>Es el programa de energía diario, con desgl...",Turbinación bombeo,3
3,Generación programada PBF Nuclear,"<p>Es el programa de energía diario, con desgl...",Nuclear,4
4,Generación programada PBF Hulla antracita Anex...,"<p>Es el programa de energía diario, con desgl...",Hulla antracita RD 134/2010,5


In [27]:
def search_wprd(df, word_to_search, column_to_search):
    '''
    This function returns the id of an indicator based on a word and a specific column
    '''
    result = df[df[column_to_search].str.contains(word_to_search)]

    return result['id'].values

search_wprd(df_indicators_list, 'nuclear', 'name')

array([ 549, 1477, 2039], dtype=int64)

### Exercise 7: Create a function that receives a list of indicators and returns the data of all of them, with each value as a single column

Test it with the following indicators: 10010, 71, 72, 74, 9, 84, 85

In [30]:
def indicator_data(indicators_list):
    '''
    This function returns the data of a list of indicators
    '''
    new_df = pd.DataFrame()

    for indicator in indicators_list:
        df_single_indicator = single_indicator(indicator, url, headers)
        new_df[indicator] = df_single_indicator

    return new_df

data = indicator_data([10010, 71, 72, 74, 9, 84, 85])

data

,10010,71,72,74,9,84,85
composited,True,False,False,False,False,False,False
disaggregated,False,False,False,False,False,False,False
geos,"[{'geo_id': 8741, 'geo_name': 'Península'}]","[{'geo_id': 8741, 'geo_name': 'Península'}]","[{'geo_id': 8741, 'geo_name': 'Península'}]","[{'geo_id': 8741, 'geo_name': 'Península'}]",[],"[{'geo_id': 8741, 'geo_name': 'Península'}]","[{'geo_id': 8741, 'geo_name': 'Península'}]"
id,10010,71,72,74,9,84,85
magnitud,"[{'name': 'Energía', 'id': 13}]","[{'name': 'Energía', 'id': 13}]","[{'name': 'Energía', 'id': 13}]","[{'name': 'Energía', 'id': 13}]","[{'name': 'Energía', 'id': 13}]","[{'name': 'Energía', 'id': 13}]","[{'name': 'Energía', 'id': 13}]"
name,Generación programada P48 Eólica,Generación programada P48 Hidráulica UGH,Generación programada P48 Hidráulica no UGH,Generación programada P48 Nuclear,Generación programada PBF Ciclo combinado,Generación programada P48 Solar fotovoltaica,Generación programada P48 Solar térmica
short_name,Eólica,Hidráulica UGH,Hidráulica no UGH,Nuclear,Ciclo combinado,Solar fotovoltaica,Solar térmica
step_type,step,step,step,step,step,step,step
tiempo,"[{'name': 'Quince minutos', 'id': 218}]","[{'name': 'Quince minutos', 'id': 218}]","[{'name': 'Quince minutos', 'id': 218}]","[{'name': 'Quince minutos', 'id': 218}]","[{'name': 'Hora', 'id': 4}]","[{'name': 'Quince minutos', 'id': 218}]","[{'name': 'Quince minutos', 'id': 218}]"
values,"[{'value': 4181.075, 'datetime': '2025-01-28T0...","[{'value': 692.125, 'datetime': '2025-01-28T00...","[{'value': 214.7, 'datetime': '2025-01-28T00:0...","[{'value': 1665.125, 'datetime': '2025-01-28T0...","[{'value': 60.0, 'datetime': '2025-01-28T00:00...","[{'value': 2.275, 'datetime': '2025-01-28T01:0...","[{'value': 1.9, 'datetime': '2025-01-28T00:00:..."
